## TODO

In [2]:
# PLAN MONDAY 
# FIX Model input Data 
# TODO Create equivalent variables but flip nodes with edges 

# DONE Understand architecture 
# DONE Understand model input data 
# DONE There are missing connections in the adjacency matrix # IDEA scrap adjacency matrix entirely if necessary? 
# DONE Create model without NeighborSampler 
# DONE Clean code 
# DONE Make model work ... 

# PLAN TUESDAY 
# TODO ADMIN - Send email with Update (ask about servers?) 
# TODO CODE - Make model more extensive 
# TODO CODE - Visualize prediction patterns and learning curve for existing model 
# TODO CODE - Create testing stage and evaluation 
# TODO THEORY - Watch GNN Theory 
# TODO THEORY - Write up Introduction 

# PLAN WEDNESDAY 
# TODO THEORY - Understand Math Theory 
# TODO THEORY - Write up methodology theory 
# TODO THEORY - Write up Literature Review 

# PLAN THURSDAY 
# TODO CODE - add statistics to a new DataFrame 
# TODO CODE - add average of transactions for accounts 

In [3]:
import pickle

import torch
import time
import random
import datetime
import numpy as np
import pandas as pd
import networkx as nx
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
from pandas import Timestamp
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
from torch_geometric.utils import negative_sampling, train_test_split_edges


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# LOADING GRAPH

In [4]:
with open("graph_data.pickle", "rb") as f:
    loaded_data = pickle.load(f)

In [9]:
input_data

Data(x=[4188, 31], edge_index=[2, 4999], edge_attr=[4999, 26], y=[4999])

In [55]:
# fraudulent transaction:
fraudulent_transaction = (labels==1).nonzero().squeeze()
fraudulent_transaction

tensor(4742)

# GRAPH NEURAL NETWORKS

In [38]:
# Define the neural network model
# class Net(torch.nn.Module):
#     def __init__(self, in_channels, hidden_channels, out_channels, heads):
#         super(Net, self).__init__()
#         self.conv1 = GCNConv(in_channels, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, out_channels)
#         self.fc = nn.Linear(out_channels, 1)

#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index)
#         x = torch.relu(x)
#         x = self.conv2(x, edge_index)
#         x = torch.relu(x)
#         x = self.fc(x)
#         return x

class GCN01(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN01, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.relu1 = nn.ReLU()
        self.conv2 = GCNConv(hidden_channels, 128)
        self.relu2 = nn.ReLU()
        self.conv3 = GCNConv(128, 256)
        self.relu3 = nn.ReLU()
        self.conv4 = GCNConv(256, 128)
        self.relu4 = nn.ReLU()
        self.conv5 = GCNConv(128, hidden_channels)
        self.relu5 = nn.ReLU()
        self.conv6 = GCNConv(hidden_channels, out_channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [39]:
print(input_data.x.size())
print(input_data.y.size())

torch.Size([4188, 31])
torch.Size([4999])


In [58]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define hyperparameters
num_epochs = 1000
hidden_channels = 128
out_channels = 64
learning_rate = 0.01

# Initialize the model, optimizer, and loss function
# model = LinkPredictor(in_channels=input_data.num_features, hidden_channels=64, out_channels=32, heads=8).to(device)
# optimizer = optim.SGD(params=model.parameters(), lr=learning_rate) # IDEA SGD = Stochastic Gradient Descent
# # criterion = torch.nn.BCELoss()
# criterion = nn.BCEWithLogitsLoss() # IDEA used for binary classification tasks --> SIGMOID + BINARY CROSS-ENTROPY LOSS

# Initialize the model, optimizer, and loss function
model = GCN01(in_channels=node_features.size(1), hidden_channels=hidden_channels, out_channels=out_channels).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

# Define the data loader
loader = DataLoader([input_data], batch_size=1, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_fraudulent_edges = 0  # Initialize total count of fraudulent edges
    fraudulent_indices = []
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        
        # Reshape the output tensor to match the shape of the target tensor
        out = out.view(-1)  # Flatten the output tensor
        
        # Ensure the dimensions of out and batch.y match
        if out.size(0) != batch.y.size(0):
            batch_size = batch.y.size(0)
            out = out[:batch_size]  # Trim the output tensor to match the batch size
        
        # Compute the binary cross-entropy loss
        loss = criterion(out, batch.y.float())  # Ensure batch.y is float type
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        # Predictions: convert logits to probabilities using sigmoid
        probabilities = torch.sigmoid(out)
        
        # Threshold probabilities to predict fraud (1) or non-fraud (0)
        predicted_labels = (probabilities > 0.5).int()  # Threshold at 0.5
        
        # Count the number of fraudulent edges (predicted as 1)
        fraudulent_edges_in_batch = (predicted_labels == 1).sum().item()
        total_fraudulent_edges += fraudulent_edges_in_batch

        # Find indices of fraudulent transactions
        batch_indices = torch.arange(len(predicted_labels))
        fraudulent_indices_batch = batch_indices[predicted_labels == 1].tolist()
        fraudulent_indices.extend(fraudulent_indices_batch)
    
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Fraudulent Edges: {total_fraudulent_edges}')
    print("Fraudulent Transaction Indices:", fraudulent_indices)


Epoch 1, Loss: 0.704785943031311, Fraudulent Edges: 2560
Fraudulent Transaction Indices: [0, 1, 4, 5, 6, 7, 8, 9, 11, 12, 14, 20, 21, 29, 30, 32, 34, 37, 38, 40, 42, 44, 45, 46, 47, 49, 53, 54, 55, 57, 58, 60, 61, 62, 64, 65, 66, 67, 68, 69, 72, 74, 77, 78, 79, 81, 85, 86, 87, 90, 91, 92, 94, 95, 96, 99, 101, 103, 107, 109, 111, 112, 115, 116, 118, 120, 122, 123, 129, 132, 134, 136, 137, 140, 142, 147, 149, 156, 157, 158, 159, 160, 164, 168, 170, 172, 174, 175, 177, 183, 185, 188, 190, 191, 192, 194, 195, 196, 199, 200, 202, 203, 205, 206, 207, 209, 211, 212, 213, 214, 215, 216, 218, 219, 221, 222, 224, 225, 227, 229, 231, 233, 235, 237, 239, 240, 241, 242, 243, 246, 248, 250, 251, 252, 254, 257, 260, 261, 264, 268, 275, 277, 284, 285, 287, 288, 289, 292, 296, 298, 300, 302, 303, 306, 307, 309, 311, 313, 316, 317, 318, 319, 320, 322, 324, 327, 328, 330, 331, 334, 335, 337, 339, 341, 342, 343, 344, 345, 346, 347, 349, 350, 351, 352, 353, 355, 356, 357, 359, 363, 365, 367, 368, 369, 370,

In [ ]:
# Assuming model is already trained and you have new data for prediction
# For demonstration purposes, let's assume you have a new graph represented as `new_input_data`

# Set model to evaluation mode
model.eval()

# Make predictions on the new data
with torch.no_grad():
    new_input_data = new_input_data.to(device)
    predicted_logits = model(new_input_data.x, new_input_data.edge_index)
    predicted_probabilities = torch.sigmoid(predicted_logits)

# Convert logits to probabilities using sigmoid
predicted_labels = (predicted_probabilities > 0.5).int()  # Threshold at 0.5

# Visualize the predictions
# For example, you could print out the predicted labels for each edge
for i, edge in enumerate(new_input_data.edge_index.t().tolist()):
    print(f'Edge {edge}: Predicted Probability: {predicted_probabilities[i].item()}, Predicted Label: {predicted_labels[i].item()}')
